In [ ]:
import tkinter as tk
import tkinter.filedialog as fd
from selenium.webdriver.common.by import By
# root = tk.Tk()
# file = fd.askopenfilename(parent=root, title='Choose a record file')
# file2 = fd.askopenfilenames(parent=root, title='Choose a pdf files')
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from IPython.display import clear_output
# root.destroy()
from time import sleep
import easygui
import time
import re
from datetime import date
from datetime import timedelta
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium import webdriver
import pandas as pd 
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import datetime
from dateutil.relativedelta import relativedelta
from selenium import webdriver
#set chromedriver.exe path
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.implicitly_wait(0)
#maximize browser
driver.maximize_window()
#launch URL
driver.get("https://www.vijaysales.com/search/air%20conditioners")

In [ ]:
driver.implicitly_wait(3)
cc=['Ceiling Fans']
# cc=['Frost Free refrigerator','Stationary Storage water heater','Colour TV','RAC (Variable speed)','TFL','LED Lamps','RAC (Fixed speed)','Direct cool refrigerator','Ceiling Fans']

list=[
# 'https://www.vijaysales.com/search/frost-free-refrigerator',
# 'https://www.vijaysales.com/search/geyser',
# 'https://www.vijaysales.com/search/tv',
# 'https://www.vijaysales.com/search/air-conditioners-inverter',
# 'https://www.vijaysales.com/search/fluorescent-tube-lights',
# 'https://www.vijaysales.com/search/led-light',
# 'https://www.vijaysales.com/search/air-conditioners-non-inverter',
# 'https://www.vijaysales.com/search/single-door-refrigerators',
'https://www.vijaysales.com/search/ceiling-fans',
]

for w in range(len(cc)):
    category=str(cc[w])
    driver.get(str(list[w]))
    
    #Get product link
    title=[]
    link=[]
    About=[]
    table=[]
    prodDescp=[]
    mainimg=[]
    
    SCROLL_PAUSE_TIME = 5

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    job=driver.find_element(by=By.CLASS_NAME, value='dvProdcts')
    soup=BeautifulSoup(job.get_attribute('innerHTML'),'html.parser')
    a=soup.find_all("a",class_="vj-cur-pnter")
    link=[]
    for job_elem in a:
    #     print(job_elem['href'])
        try:
            link.append(job_elem['href'])
        except:
            pass
    link = [*set(link)]
    df = pd.DataFrame({
    'Link': link})

    #Get images link
    for j in range(0,len(df)):
        print(j)
        driver.get(df['Link'][j])
        time.sleep(1)
        
        #title
        job=driver.find_element(by=By.CLASS_NAME, value='vs-wrapper')
        soup=BeautifulSoup(job.get_attribute('innerHTML'),'html.parser')
        tt=soup.find("h1",id="ContentPlaceHolder1_h1ProductTitle").text
        title.append(tt)
    
        while True:
            try:
                driver.find_element(by=By.CLASS_NAME, value='slick-next').click()
            except:
                break
        try:
            img=[]
            job=driver.find_element(by=By.CLASS_NAME, value='vs-wrapper')
            soup=BeautifulSoup(job.get_attribute('innerHTML'),'html.parser')
            a=soup.find_all("img",class_="img-responsive w-100 lazy clsprodImgs")
            for item in a:
        #         print((item['src']).replace("_182x182", ""))
                img.append(item['src'].replace("_182x182", ""))
        except:
            img.append('none')
        mainimg.append(img)
                
        try:
            #bullet points
            b=soup.find("div",class_="clsKeyFeatures").text
            About.append(b)
        except:
            About.append('none')
        try:
            #table
            b=soup.find_all("div",class_="specification col_6")[1].text
            table.append(b)
        except:
            table.append('none')
        try:
            #prodDescp
            b=soup.find("div",class_="specification col_6").text
            prodDescp.append(b)
        except:
            prodDescp.append('none')    
            
        
    df['Title']=0
    df['bullet points']=0
    df['table']=0
    df['prodDescp']=0
    df['image link']=0
    df['Title']=title
    df['bullet points']=About
    df['table']=table
    df['prodDescp']=prodDescp
    df['image link']=mainimg
    df.to_csv('final'+str(category)+'.csv')
#     break